In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
#Preparation

In [ ]:
con = duckdb.connect('kia.db')

In [ ]:
con.sql("create table peserta as select * from Kepesertaan_2023.csv")

In [ ]:
con.sql("create table fkrtl as select * from FKRTL_2023.csv")

In [ ]:
con.sql("create table fktp as select * from read_csv_auto('FKTP_2023.csv', ALL_VARCHAR=TRUE)")

In [ ]:
con.sql("create table fkrtl_sek as select * from FKRTL_Sekunder_2023.csv")

In [ ]:
con.sql("create table nk as select * from Non_Kapitasi_2023.csv")

In [ ]:
con.sql("CREATE TABLE nk1 AS SELECT PSTV01, PNK03, PNK05, PNK06, PNK12, PNK13A, PNK14 FROM nk")

In [ ]:
con.sql("CREATE TABLE fktp1 AS SELECT PSTV01, FKP02, FKP03, FKP04, FKP05, FKP12, FKP13, FKP14A, FKP15, FKP21 FROM fktp")

In [ ]:
con.sql("CREATE TABLE fkrtl1 AS SELECT PSTV01, FKP02, FKL02, FKL03, FKL04, FKL05, FKL09, FKL11, FKL12, FKL14, FKL15A, FKL16, FKL17A, FKL18 FROM fkrtl")

In [ ]:
con.sql("CREATE TABLE sek1 AS SELECT FKL02, FKL24, FKL24A FROM fkrtl_sek")

In [ ]:
con.sql("CREATE TABLE peserta1 AS SELECT PSTV01, PSTV03, PSTV06, PSTV18 FROM peserta")

In [ ]:
con.sql("CREATE TABLE rs AS SELECT p.*, f.* FROM fkrtl1 AS p LEFT JOIN sek1 AS f ON p.FKL02 = f.fkl02")

In [ ]:
con.sql("""
CREATE TABLE klin AS 
SELECT 
    COALESCE(f.PSTV01, p.PSTV01) AS PSTV01, 
    COALESCE(f.FKP02, p.FKP02) AS FKP02, 
    f.*, 
    p.*
FROM rs AS f 
FULL OUTER JOIN fktp1 AS p 
ON f.PSTV01 = p.PSTV01 AND f.FKP02 = p.FKP02;
""")


In [ ]:
con.sql("create table kia as select f.*,p.* from klin as f LEFT JOIN peserta1 as p on f.PSTV01=p.PSTV01")

In [ ]:
# Remove column duplicates
con.sql('ALTER TABLE kia DROP COLUMN "pstv01:1"')
con.sql('ALTER TABLE kia DROP COLUMN "fkp02:1"')
con.sql("ALTER TABLE kia DROP COLUMN fkl12")
con.sql("ALTER TABLE kia DROP COLUMN fkl11")
con.sql("ALTER TABLE kia DROP COLUMN fkl05")
con.sql("ALTER TABLE kia DROP COLUMN fkl14")
con.sql('ALTER TABLE kia DROP COLUMN "fkl02:1"')
con.sql('ALTER TABLE kia DROP COLUMN "pstv01:2"')
con.sql('ALTER TABLE kia DROP COLUMN "fkp02:2"')
con.sql("ALTER TABLE kia DROP COLUMN fkp05")
con.sql("ALTER TABLE kia DROP COLUMN fkp12")
con.sql("ALTER TABLE kia DROP COLUMN fkp13")
con.sql("ALTER TABLE kia DROP COLUMN fkp21")
con.sql('ALTER TABLE kia DROP COLUMN "pstv01:3"')
con.sql("ALTER TABLE kia DROP COLUMN pstv03")
con.sql("ALTER TABLE kia DROP COLUMN pstv06")

Pre-Processing

In [ ]:
con.sql("alter table kia add column age int")

In [ ]:
#Generate Age
con.sql("""
UPDATE kia 
SET age = CASE 
            WHEN fkl03 IS NOT NULL THEN (CAST(right(fkl03, 4) AS INTEGER) - CAST(right(pstv03, 4) AS INTEGER))
            WHEN fkp03 IS NOT NULL THEN (CAST(right(fkp03, 4) AS INTEGER) - CAST(right(pstv03, 4) AS INTEGER))
          END;
""")

In [ ]:
#Generate Region of Residence 
con.sql("ALTER TABLE kia ADD COLUMN dom INT")
con.sql("""
UPDATE kia 
SET dom = CASE 
            WHEN fkl05 IN ('JAWA TENGAH','JAWA TIMUR','BANTEN', 'DAERAH ISTIMEWA YOGYAKARTA', 'DKI JAKARTA', 'JAWA BARAT') THEN 0
            WHEN fkp05 IN ('JAWA TENGAH','JAWA TIMUR','BANTEN', 'DAERAH ISTIMEWA YOGYAKARTA', 'DKI JAKARTA', 'JAWA BARAT') THEN 0
            ELSE 1
          END;
""")


In [ ]:
con.sql("alter table kia add column outcome int")

In [ ]:
#Generate outcome
con.sql("""
UPDATE kia 
SET outcome = CASE
               WHEN fkl14 IN ('Meninggal', 'MENINGGAL') OR fkp13 IN ('Meninggal', 'MENINGGAL') THEN 1
               WHEN fkl14 IS NULL AND fkp13 IS NULL THEN NULL
               ELSE 0
             END;
""")


In [ ]:
con.sql("alter table kia add column subsid int")

In [ ]:
#Generate subsidy membership status
con.sql("""
UPDATE kia 
SET subsid = CASE 
            WHEN fkl12 IN ('PBI APBN', 'PBI APBD') THEN 0
            WHEN fkl12 IS NULL AND fkp12 IN ('PBI APBN', 'PBI APBD') THEN 0
            WHEN fkl12 IS NULL AND fkp12 IS NULL THEN NULL
            ELSE 1
          END;
""")


In [ ]:
con.sql("ALTER TABLE kia DROP COLUMN pstv011")

In [ ]:
con.sql("alter table kia add column dol int")

In [ ]:
df = con.sql("select * from kia").df()

In [ ]:
df.to_csv("pregnancy new.csv",index=False)

In [ ]:
df = pd.read_csv("pregnancy new.csv")

In [ ]:
df

In [ ]:
# Convert to datetime
df['FKL03'] = pd.to_datetime(df['FKL03'], format='%d%b%Y', errors='coerce')
df['FKL04'] = pd.to_datetime(df['FKL04'], format='%d%b%Y', errors='coerce')
df['FKP03'] = pd.to_datetime(df['FKP03'], format='%d%b%Y', errors='coerce')
df['FKP04'] = pd.to_datetime(df['FKP04'], format='%d%b%Y', errors='coerce')

In [ ]:
# Buat kolom baru 'combined_date' yang berisi nilai lebih awal antara FKP03 dan FKL03 jika keduanya ada
df['combined_date'] = df[['FKP03', 'FKL03']].min(axis=1)

In [ ]:
# Lakukan sorting berdasarkan PSTV01 dan kolom combined_date
df = df.sort_values(by=['PSTV01', 'combined_date'])

In [ ]:
# Daftar kode diagnosis yang diperhatikan di semua kolom diagnosis
diagnosis_codes = ['O80', 'O81', 'O82', 'O83', 'O84', 'O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O346']

# Buat kolom 'dol' berdasarkan kondisi, dan tetap gunakan tipe datetime
df['dol'] = np.where(
    df['FKP14A'].isin(diagnosis_codes) |
    df['FKP15'].isin(diagnosis_codes), df['combined_date'],  
    np.where(
        df['FKL15A'].isin(diagnosis_codes) |  
        df['FKL17A'].isin(diagnosis_codes) | 
        df['fkl24a'].isin(diagnosis_codes) |
        df['FKL16'].isin(diagnosis_codes) |
        df['FKL18'].isin(diagnosis_codes) |
        df['fkl24'].isin(diagnosis_codes), df['combined_date'],  
        pd.NaT  
    )
)

# Konversi kolom 'dol' menjadi datetime
df['dol'] = pd.to_datetime(df['dol'])


In [ ]:
df[df['dol'].notna()]

In [ ]:
#CHECKPOINT
df.to_csv("pregnancy new.csv",index=False)

In [ ]:
# 1. Urutkan data berdasarkan 'PSTV01' dan 'dol' terlebih dahulu
df = df.sort_values(by=['PSTV01', 'dol'])

# 2. Hitung selisih tanggal antar 'dol' yang berurutan untuk setiap 'PSTV01'
df['dol_diff'] = df.groupby('PSTV01')['dol'].diff().dt.days  # Hitung selisih dalam satuan hari

# 3. Tandai baris yang berjarak < 2 bulan (60 hari)
df['close_dol'] = df['dol_diff'] < 60

# 4. Filter data untuk mendapatkan 'dol' pertama dari setiap 'PSTV01' yang memiliki jarak antar 'dol' < 2 bulan
# 'first' akan otomatis memilih 'dol' pertama dalam kelompok
filtered_df = df.loc[df.groupby('PSTV01')['close_dol'].transform('first') == False]

# 5. Lihat hasil yang sudah difilter
print(filtered_df[['PSTV01', 'dol', 'dol_diff', 'close_dol']])

In [ ]:
import pandas as pd
import numpy as np

# Urutkan DataFrame berdasarkan 'PSTV01' dan 'combined_date'
df = df.sort_values(['PSTV01', 'combined_date'])

# Inisialisasi kolom 'tgl2' (tanggal kehamilan yang valid) dengan NaT
df['tgl2'] = pd.NaT

# Dictionary untuk menyimpan tanggal diagnosis terakhir yang tidak berdekatan untuk masing-masing 'PSTV01'
last_valid_diagnosis = {}

# Iterasi setiap baris untuk menetapkan 'tgl2'
for index, row in df.iterrows():
    pstv01 = row['PSTV01']
    dol = row['dol']

    # Hanya proses baris yang memiliki 'dol' yang valid (bukan NaN)
    if pd.notnull(dol):
        if pstv01 in last_valid_diagnosis:
            # Hitung selisih hari dari dol saat ini ke dol terakhir yang valid
            selisih_hari = (dol - last_valid_diagnosis[pstv01]).days
            if selisih_hari > 280:  # Jika selisih hari lebih dari 280, artinya ini kehamilan baru
                df.at[index, 'tgl2'] = dol
                last_valid_diagnosis[pstv01] = dol  # Perbarui tanggal diagnosis terakhir yang valid
        else:
            # Jika 'pstv01' belum ada di dictionary, ini adalah diagnosis awal yang valid
            df.at[index, 'tgl2'] = dol
            last_valid_diagnosis[pstv01] = dol

# Hitung jumlah kehamilan (n_grav) berdasarkan tgl2 yang valid
current_pstv01 = None
pregnancy_count = 0

for index, row in df.iterrows():
    # Cek apakah masih pasien yang sama
    if row['PSTV01'] != current_pstv01:
        # Reset hitungan jika ganti pasien baru
        current_pstv01 = row['PSTV01']
        pregnancy_count = 1 if pd.notnull(row['tgl2']) else 0
    else:
        # Jika 'tgl2' valid dan berbeda dari diagnosis sebelumnya, hitung sebagai kehamilan baru
        if pd.notnull(row['tgl2']) and row['tgl2'] != df.at[index - 1, 'tgl2']:
            pregnancy_count += 1

    # Tetapkan hitungan kehamilan ke baris saat ini
    df.at[index, 'n_grav'] = pregnancy_count

# Tampilkan hasil
print(df[['PSTV01', 'combined_date', 'dol', 'tgl2', 'n_grav']])


In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['dol_diff', 'valid_dol', 'n_grav']

# Hapus kolom yang tidak diperlukan
df = df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
#CHECKPOINT
df.to_csv("pregnancy new.csv",index=False)

In [ ]:
df = pd.read_csv("pregnancy new.csv")

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['dol', 'tgl2', 'n_grav']

In [ ]:
# Daftar kode diagnosis yang diperhatikan di semua kolom diagnosis
diagnosis_codes = ['O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07']

# Buat kolom 'dol' berdasarkan kondisi, dan tetap gunakan tipe datetime
df['doa'] = np.where(
    df['FKP14A'].isin(diagnosis_codes) |
    df['FKP15'].isin(diagnosis_codes), df['combined_date'],  # Cek diagnosis di puskesmas
    np.where(
        df['FKL15A'].isin(diagnosis_codes) |  # Cek diagnosis di RS (FKL15A)
        df['FKL17A'].isin(diagnosis_codes) |  # Cek diagnosis di RS (FKL17A)
        df['fkl24a'].isin(diagnosis_codes) |
        df['FKL16'].isin(diagnosis_codes) |
        df['FKL18'].isin(diagnosis_codes) |
        df['fkl24'].isin(diagnosis_codes), df['combined_date'],  # Cek diagnosis di RS (FKL24A)
        pd.NaT  # Jika tidak ada diagnosis O80-O84, isi dengan NaT (Not a Time)
    )
)

# Konversi kolom 'dol' menjadi datetime
df['doa'] = pd.to_datetime(df['doa'])

# Urutkan DataFrame berdasarkan 'PSTV01' dan 'combined_date'
df = df.sort_values(['PSTV01', 'combined_date'])

# Inisialisasi kolom 'tgl2' (tanggal kehamilan yang valid) dengan NaT
df['fin_a'] = pd.NaT

# Dictionary untuk menyimpan tanggal diagnosis terakhir yang tidak berdekatan untuk masing-masing 'PSTV01'
last_valid_diagnosis = {'doa': {}}

# Iterasi setiap baris untuk menetapkan 'tgl2'
for index, row in df.iterrows():
    pstv01 = row['PSTV01']
    doa = row['doa']


    # Proses untuk kolom 'doa'
    if pd.notnull(doa):
        if pstv01 in last_valid_diagnosis['doa']:
            # Hitung selisih hari untuk 'doa'
            selisih_hari_doa = (doa - last_valid_diagnosis['doa'][pstv01]).days
            if selisih_hari_doa > 60:  # Jika selisih hari lebih dari 60, artinya valid untuk 'doa'
                df.at[index, 'fin_a'] = doa
                last_valid_diagnosis['doa'][pstv01] = doa  # Perbarui tanggal diagnosis terakhir yang valid untuk 'doa'
        else:
            # Jika 'pstv01' belum ada di dictionary untuk 'doa', ini adalah diagnosis awal yang valid
            df.at[index, 'fin_a'] = doa
            last_valid_diagnosis['doa'][pstv01] = doa

In [ ]:
# Daftar kode diagnosis yang diperhatikan di semua kolom diagnosis
diagnosis_codes = ['O80', 'O81', 'O82', 'O83', 'O84']

# Buat kolom 'dol' berdasarkan kondisi, dan tetap gunakan tipe datetime
df['dop'] = np.where(
    df['FKP14A'].isin(diagnosis_codes) |
    df['FKP15'].isin(diagnosis_codes), df['combined_date'],  # Cek diagnosis di puskesmas
    np.where(
        df['FKL15A'].isin(diagnosis_codes) |  # Cek diagnosis di RS (FKL15A)
        df['FKL17A'].isin(diagnosis_codes) |  # Cek diagnosis di RS (FKL17A)
        df['fkl24a'].isin(diagnosis_codes) |
        df['FKL16'].isin(diagnosis_codes) |
        df['FKL18'].isin(diagnosis_codes) |
        df['fkl24'].isin(diagnosis_codes), df['combined_date'],  # Cek diagnosis di RS (FKL24A)
        pd.NaT  # Jika tidak ada diagnosis O80-O84, isi dengan NaT (Not a Time)
    )
)

# Konversi kolom 'dol' menjadi datetime
df['dop'] = pd.to_datetime(df['dop'])

# Urutkan DataFrame berdasarkan 'PSTV01' dan 'combined_date'
df = df.sort_values(['PSTV01', 'combined_date'])

# Inisialisasi kolom 'tgl2' (tanggal kehamilan yang valid) dengan NaT
df['fin_p'] = pd.NaT

# Dictionary untuk menyimpan tanggal diagnosis terakhir yang tidak berdekatan untuk masing-masing 'PSTV01'
last_valid_diagnosis = {'dop': {}}

# Iterasi setiap baris untuk menetapkan 'tgl2'
for index, row in df.iterrows():
    pstv01 = row['PSTV01']
    dop = row['dop']

    # Proses untuk kolom 'dop'
    if pd.notnull(dop):
        if pstv01 in last_valid_diagnosis['dop']:
            # Hitung selisih hari untuk 'dop'
            selisih_hari_dop = (dop - last_valid_diagnosis['dop'][pstv01]).days
            if selisih_hari_dop > 280:  # Jika selisih hari lebih dari 280, artinya ini kehamilan baru
                df.at[index, 'fin_p'] = dop
                last_valid_diagnosis['dop'][pstv01] = dop  # Perbarui tanggal diagnosis terakhir yang valid untuk 'dop'
        else:
            # Jika 'pstv01' belum ada di dictionary untuk 'dop', ini adalah diagnosis awal yang valid
            df.at[index, 'fin_p'] = dop
            last_valid_diagnosis['dop'][pstv01] = dop

In [ ]:
# Urutkan DataFrame berdasarkan 'PSTV01' dan 'combined_date'
df = df.sort_values(['PSTV01', 'combined_date'])

# Gabungkan kolom 'fin_a' dan 'fin_p', prioritaskan 'fin_p'
df['dopt'] = df['fin_p'].combine_first(df['fin_a'])


In [ ]:
# Urutkan DataFrame berdasarkan 'PSTV01' dan 'combined_date'
df = df.sort_values(['PSTV01', 'combined_date'])

# Gabungkan kolom 'fin_a' dan 'fin_p', prioritaskan 'fin_p'
df['dopt'] = df['fin_p'].combine_first(df['fin_a'])

# Inisialisasi kolom 'fin_g' dengan NaT
df['fin_g'] = pd.NaT

# Dictionary untuk menyimpan tanggal terakhir yang valid untuk masing-masing 'PSTV01'
last_valid_date = {}

# Iterasi setiap baris untuk menetapkan 'fin_g'
for index, row in df.iterrows():
    pstv01 = row['PSTV01']
    dopt = row['dopt']

    # Proses untuk kolom 'dopt'
    if pd.notnull(dopt):  # Hanya proses baris dengan dopt yang valid
        if pstv01 in last_valid_date:  # Jika PSTV01 sudah ada di dictionary
            # Hitung selisih hari antara dopt saat ini dan yang terakhir
            selisih_hari = (dopt - last_valid_date[pstv01]).days
            if selisih_hari > 180:  # Jika selisih hari > 180, valid untuk kehamilan baru
                df.at[index, 'fin_g'] = dopt  # Tetapkan dopt ke fin_g
                last_valid_date[pstv01] = dopt  # Perbarui tanggal terakhir yang valid
        else:
            # Jika PSTV01 belum ada di dictionary, tambahkan sebagai tanggal pertama yang valid
            df.at[index, 'fin_g'] = dopt
            last_valid_date[pstv01] = dopt


In [ ]:
## Menandai duplikat pada kolom 'dopt' berdasarkan 'PSTV01'
#df['dopt'] = df.groupby('PSTV01')['dopt'].apply(lambda x: x.mask(x.duplicated(), pd.NaT))

In [ ]:
# Inisialisasi kolom n_preg dengan nilai NaT
df['n_preg'] = pd.NaT

# Mengisi n_preg hanya untuk baris dengan dopt yang valid
df.loc[df['fin_g'].notnull(), 'n_preg'] = (
    df[df['fin_g'].notnull()]
    .groupby('PSTV01')
    .cumcount() + 1  # Hitung urutan mulai dari 1
)

In [ ]:
# Get unique values and their counts in the 'n_grav' column
value_counts = df['n_preg'].value_counts()

# Display the result
print(value_counts)

In [ ]:
# Filter baris dengan n_preg > 2
filtered_df = df[df['n_preg'] > 9]

In [ ]:
filtered_df

In [ ]:
# Pilih kolom yang ingin ditampilkan
columns_to_view = [
    'FKL15A', 'FKL17A', 'FKL16', 'fkl24a', 
    'FKL16', 'FKL18', 'fkl24', 'FKP14A', 'FKP15'
]

# Tampilkan subset DataFrame berdasarkan kolom yang diinginkan
filtered_subset = filtered_df[columns_to_view]


In [ ]:
filtered_subset

In [ ]:
tes = df[df['PSTV01'] == 32850288]

In [ ]:
tes

In [ ]:
#CHECKPOINT
df.to_csv("pregnancy new.csv",index=False)

In [ ]:
df = pd.read_csv("pregnancy new.csv")

In [ ]:
del df

In [ ]:
#Cohort untuk masing2 kehamilan

In [ ]:
#PRIMO

In [1]:
import pandas as pd
import duckdb
import numpy as np

In [2]:
# Membuat tabel 'primi' sebagai salinan dari 'df'
primi = pd.read_csv("pregnancy new.csv")

In [3]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 1 untuk setiap PSTV01
ref_values = primi.loc[primi['n_preg'] == 1, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
primi['ref'] = primi['PSTV01'].map(ref_values['dopt'])

In [4]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
primi = primi.drop(columns=columns_to_drop, errors='ignore')

In [5]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
primi['combined_date'] = pd.to_datetime(primi['combined_date'], errors='coerce')
primi['ref'] = pd.to_datetime(primi['ref'], errors='coerce')

In [6]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    primi[f'b_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & (primi['combined_date'] <= primi['ref'])  # Syarat tambahan
    ).astype(int)


In [7]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    primi[f'b_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & 
        ((primi['ref'] - primi['combined_date']).dt.days > 30)
    ).astype(int)

C:\Users\User\AppData\Local\Temp\ipykernel_12304\4169253800.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'b_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\4169253800.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'b_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\4169253800.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [8]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    primi[f'b_{condition}'] = (
      (  
        primi['FKP14A'].str.contains(regex, na=False) |
        primi['FKL15A'].str.contains(regex, na=False) |
        primi['FKL17A'].str.contains(regex, na=False) |
        primi['fkl24a'].str.contains(regex, na=False)
      ) & (primi['combined_date'] <= primi['ref'])  # Syarat tambahan
    ).astype(int)


C:\Users\User\AppData\Local\Temp\ipykernel_12304\232310690.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKP14A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\232310690.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKL15A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\232310690.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKL17A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\232310690.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['fkl24a'].str.contains(regex, na=False)
C:\Use

In [9]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    primi[f'c_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & 
        ((primi['combined_date'] - primi['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

C:\Users\User\AppData\Local\Temp\ipykernel_12304\2214914892.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'c_{condition}'] = (


In [10]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    primi[f'c_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & 
        ((primi['combined_date'] - primi['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

C:\Users\User\AppData\Local\Temp\ipykernel_12304\595263366.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'c_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\595263366.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'c_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\595263366.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

In [11]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    primi[f'a_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & (primi['combined_date'] > primi['ref'])  # Syarat tambahan
    ).astype(int)


C:\Users\User\AppData\Local\Temp\ipykernel_12304\547326129.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'a_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\547326129.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'a_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\547326129.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

In [12]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    primi[f'a_{condition}'] = (
      (  
        primi['FKP14A'].str.contains(regex, na=False) |
        primi['FKL15A'].str.contains(regex, na=False) |
        primi['FKL17A'].str.contains(regex, na=False) |
        primi['fkl24a'].str.contains(regex, na=False)
      ) & (primi['combined_date'] > primi['ref'])  # Syarat tambahan
    ).astype(int)


C:\Users\User\AppData\Local\Temp\ipykernel_12304\2455952121.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKP14A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\2455952121.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKL15A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\2455952121.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['FKL17A'].str.contains(regex, na=False) |
C:\Users\User\AppData\Local\Temp\ipykernel_12304\2455952121.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  primi['fkl24a'].str.contains(regex, na=False)
C:

In [13]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    primi[f'a_{condition}'] = (
        (
            primi['FKP14A'].isin(codes) |
            primi['FKL15A'].isin(codes) |
            primi['FKL17A'].isin(codes) |
            primi['fkl24a'].isin(codes)
        ) & 
        ((primi['combined_date'] - primi['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

C:\Users\User\AppData\Local\Temp\ipykernel_12304\3501592884.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'a_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\3501592884.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi[f'a_{condition}'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_12304\3501592884.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [17]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
primi['PSTV18'] = primi['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
primi['ref_year'] = primi['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
primi['mort'] = (primi['PSTV18'] == primi['ref_year']).astype(int)
primi['1ymort'] = (abs(primi['PSTV18'] - primi['ref_year']) == 1).astype(int)



C:\Users\User\AppData\Local\Temp\ipykernel_12304\1292230122.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  primi['ref_year'] = primi['ref'].dt.year


In [ ]:
primi.to_csv("primi.csv",index=False)

In [18]:
##Washout period

# Hapus baris dengan ref_year 2015 dan hitung jumlah baris yang dihapus
rows_to_drop = primi[primi['ref_year'] == 2015]
num_rows_dropped = len(rows_to_drop)

# Update DataFrame
primi_wash = primi[primi['ref_year'] != 2015]

print(f"Jumlah baris yang dihapus: {num_rows_dropped}")

Jumlah baris yang dihapus: 675550


In [15]:
primi

,PSTV01,FKL15A,FKL17A,fkl24a,FKP14A,PSTV18,age,dom,outcome,subsid,...,a_obstrau,a_pph,a_retained,a_normal,a_instrum,a_caesar,a_assisted,a_multiple,mort,1ymort
0,802,Z01,O80,O48,NaN,0,25,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,802,Z01,O80,O42,NaN,0,25,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,802,NaN,NaN,NaN,Z36,0,27,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,802,NaN,NaN,NaN,Z36,0,27,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,802,NaN,NaN,NaN,Z36,0,27,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533612,479436606,O14,O99,O82,NaN,0,18,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7533613,479436606,O14,O99,O99,NaN,0,18,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7533614,479436606,O14,O99,O15,NaN,0,18,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7533615,479436606,O14,O99,Z37,NaN,0,18,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Hapus kolom 'ref_year' setelah selesai digunakan
primi_wash.drop(columns=['ref_year'], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_12304\2937647601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primi_wash.drop(columns=['ref_year'], inplace=True)


In [20]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A', 'PSTV18']

# Hapus kolom yang tidak diperlukan
primi_wash = primi_wash.drop(columns=columns_to_drop, errors='ignore')

In [21]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
ichi = primi_wash.drop(columns=columns_to_drop, errors='ignore')

In [23]:
ichi

,PSTV01,age,dom,outcome,subsid,n_preg,b_typhoid,b_cholera,b_v_age,b_b_age,...,a_obstrau,a_pph,a_retained,a_normal,a_instrum,a_caesar,a_assisted,a_multiple,mort,1ymort
0,802,25,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,802,25,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,802,27,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,802,27,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,802,27,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533612,479436606,18,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7533613,479436606,18,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7533614,479436606,18,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7533615,479436606,18,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Set all values in the 'n_preg' column to 1
ichi['n_preg'] = 1

In [24]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'
}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in ichi.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_ich = ichi.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
# Set all values in the 'n_preg' column to 1
fin_ich['n_preg'] = 1

In [25]:
fin_ich

,PSTV01,subsid,age,dom,n_preg,outcome,mort,1ymort,b_typhoid,b_cholera,...,a_umbilical,a_laceration,a_obstrau,a_pph,a_retained,a_normal,a_instrum,a_caesar,a_assisted,a_multiple
0,802,1,25,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1101,0,37,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1664,1,23,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1859,1,24,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,2693,1,16,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389244,479197256,0,25,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389245,479248146,0,29,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389246,479399959,1,23,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389247,479436606,0,18,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
fin_ich.to_csv("1st_washed.csv",index=False)

In [ ]:
##loro

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
secon = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 2
valid_pstv01 = secon.loc[secon['n_preg'] == 2, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
secon = secon[secon['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = secon.loc[secon['n_preg'] == 2, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
secon['ref'] = secon['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
secon = secon.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
secon['combined_date'] = pd.to_datetime(secon['combined_date'], errors='coerce')
secon['ref'] = pd.to_datetime(secon['ref'], errors='coerce')

In [ ]:
# Dictionary untuk RPD
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    secon[f'b_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & (secon['combined_date'] <= secon['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    secon[f'b_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & 
        ((secon['ref'] - secon['combined_date']).dt.days > 30)
    ).astype(int)

In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    secon[f'b_{condition}'] = (
      (  
        secon['FKP14A'].str.contains(regex, na=False) |
        secon['FKL15A'].str.contains(regex, na=False) |
        secon['FKL17A'].str.contains(regex, na=False) |
        secon['fkl24a'].str.contains(regex, na=False)
      ) & (secon['combined_date'] <= secon['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    secon[f'c_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & 
        ((secon['combined_date'] - secon['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    secon[f'c_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & 
        ((secon['combined_date'] - secon['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}


# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    secon[f'a_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & (secon['combined_date'] > secon['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    secon[f'a_{condition}'] = (
      (  
        secon['FKP14A'].str.contains(regex, na=False) |
        secon['FKL15A'].str.contains(regex, na=False) |
        secon['FKL17A'].str.contains(regex, na=False) |
        secon['fkl24a'].str.contains(regex, na=False)
      ) & (secon['combined_date'] > secon['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    secon[f'a_{condition}'] = (
        (
            secon['FKP14A'].isin(codes) |
            secon['FKL15A'].isin(codes) |
            secon['FKL17A'].isin(codes) |
            secon['fkl24a'].isin(codes)
        ) & 
        ((secon['combined_date'] - secon['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
secon = secon.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
secon['PSTV18'] = secon['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
secon['ref_year'] = secon['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel secon
secon['mort'] = (secon['PSTV18'] == secon['ref_year']).astype(int)
secon['1ymort'] = (abs(secon['PSTV18'] - secon['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
secon.drop(columns=['ref_year'], inplace=True)

In [ ]:
secon.to_csv("secon.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
nii = secon.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'

}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in nii.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_nii = nii.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_nii

In [ ]:
fin_nii.to_csv("2nd.csv",index=False)

In [ ]:
fin_nii = pd.read_csv("2nd.csv")

In [ ]:
fin_nii['n_preg'] = 2

In [ ]:
fin_nii

In [ ]:
#telu

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
trio = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 3
valid_pstv01 = trio.loc[trio['n_preg'] == 3, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
trio = trio[trio['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = trio.loc[trio['n_preg'] == 3, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
trio['ref'] = trio['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
trio = trio.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
trio['combined_date'] = pd.to_datetime(trio['combined_date'], errors='coerce')
trio['ref'] = pd.to_datetime(trio['ref'], errors='coerce')

In [ ]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'schizaff': ['F30', 'F31', 'F32', 'F33', 'F34', 'F38', 'F39'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': [ ],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    trio[f'b_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & (trio['combined_date'] <= trio['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    trio[f'b_{condition}'] = (
      (  
        trio['FKP14A'].str.contains(regex, na=False) |
        trio['FKL15A'].str.contains(regex, na=False) |
        trio['FKL17A'].str.contains(regex, na=False) |
        trio['fkl24a'].str.contains(regex, na=False)
      ) & (trio['combined_date'] <= trio['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    trio[f'b_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & 
        ((trio['ref'] - trio['combined_date']).dt.days > 30)
    ).astype(int)

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    trio[f'c_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & 
        ((trio['combined_date'] - trio['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    trio[f'c_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & 
        ((trio['combined_date'] - trio['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    trio[f'a_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & (trio['combined_date'] > trio['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    trio[f'a_{condition}'] = (
      (  
        trio['FKP14A'].str.contains(regex, na=False) |
        trio['FKL15A'].str.contains(regex, na=False) |
        trio['FKL17A'].str.contains(regex, na=False) |
        trio['fkl24a'].str.contains(regex, na=False)
      ) & (trio['combined_date'] > trio['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    trio[f'a_{condition}'] = (
        (
            trio['FKP14A'].isin(codes) |
            trio['FKL15A'].isin(codes) |
            trio['FKL17A'].isin(codes) |
            trio['fkl24a'].isin(codes)
        ) & 
        ((trio['combined_date'] - trio['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
trio = trio.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
trio['PSTV18'] = trio['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
trio['ref_year'] = trio['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
trio['mort'] = (trio['PSTV18'] == trio['ref_year']).astype(int)
trio['1ymort'] = (abs(trio['PSTV18'] - trio['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
trio.drop(columns=['ref_year'], inplace=True)


In [ ]:
trio.to_csv("trio.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
san = trio.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'

}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in san.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_san = san.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_san

In [ ]:
fin_san.to_csv("3rd.csv",index=False)

In [ ]:
#papat

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
quat = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 3
valid_pstv01 = quat.loc[quat['n_preg'] == 4, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
quat = quat[quat['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = quat.loc[quat['n_preg'] == 4, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
quat['ref'] = quat['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
quat = quat.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
quat['combined_date'] = pd.to_datetime(quat['combined_date'], errors='coerce')
quat['ref'] = pd.to_datetime(quat['ref'], errors='coerce')

In [ ]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    quat[f'b_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & (quat['combined_date'] <= quat['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    quat[f'b_{condition}'] = (
      (  
        quat['FKP14A'].str.contains(regex, na=False) |
        quat['FKL15A'].str.contains(regex, na=False) |
        quat['FKL17A'].str.contains(regex, na=False) |
        quat['fkl24a'].str.contains(regex, na=False)
      ) & (quat['combined_date'] <= quat['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quat[f'b_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & 
        ((quat['ref'] - quat['combined_date']).dt.days > 30)
    ).astype(int)

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quat[f'c_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & 
        ((quat['combined_date'] - quat['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quat[f'c_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & 
        ((quat['combined_date'] - quat['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    quat[f'a_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & (quat['combined_date'] > quat['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    quat[f'a_{condition}'] = (
      (  
        quat['FKP14A'].str.contains(regex, na=False) |
        quat['FKL15A'].str.contains(regex, na=False) |
        quat['FKL17A'].str.contains(regex, na=False) |
        quat['fkl24a'].str.contains(regex, na=False)
      ) & (quat['combined_date'] > quat['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quat[f'a_{condition}'] = (
        (
            quat['FKP14A'].isin(codes) |
            quat['FKL15A'].isin(codes) |
            quat['FKL17A'].isin(codes) |
            quat['fkl24a'].isin(codes)
        ) & 
        ((quat['combined_date'] - quat['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
quat = quat.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
quat['PSTV18'] = quat['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
quat['ref_year'] = quat['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
quat['mort'] = (quat['PSTV18'] == quat['ref_year']).astype(int)
quat['1ymort'] = (abs(quat['PSTV18'] - quat['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
quat.drop(columns=['ref_year'], inplace=True)


In [ ]:
quat.to_csv("quat.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref']

# Hapus kolom yang tidak diperlukan
yon = quat.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'
}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in yon.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_yon = yon.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_yon

In [ ]:
fin_yon.to_csv("4th.csv",index=False)

In [ ]:
#limo

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
quin = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 3
valid_pstv01 = quin.loc[quin['n_preg'] == 5, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
quin = quin[quin['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = quin.loc[quin['n_preg'] == 5, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
quin['ref'] = quin['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
quin = quin.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
quin['combined_date'] = pd.to_datetime(quin['combined_date'], errors='coerce')
quin['ref'] = pd.to_datetime(quin['ref'], errors='coerce')

In [ ]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    quin[f'b_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & (quin['combined_date'] <= quin['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    quin[f'b_{condition}'] = (
      (  
        quin['FKP14A'].str.contains(regex, na=False) |
        quin['FKL15A'].str.contains(regex, na=False) |
        quin['FKL17A'].str.contains(regex, na=False) |
        quin['fkl24a'].str.contains(regex, na=False)
      ) & (quin['combined_date'] <= quin['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quin[f'b_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & 
        ((quin['ref'] - quin['combined_date']).dt.days > 30)
    ).astype(int)

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quin[f'c_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & 
        ((quin['combined_date'] - quin['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quin[f'c_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & 
        ((quin['combined_date'] - quin['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    quin[f'a_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & (quin['combined_date'] > quin['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    quin[f'a_{condition}'] = (
      (  
        quin['FKP14A'].str.contains(regex, na=False) |
        quin['FKL15A'].str.contains(regex, na=False) |
        quin['FKL17A'].str.contains(regex, na=False) |
        quin['fkl24a'].str.contains(regex, na=False)
      ) & (quin['combined_date'] > quin['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    quin[f'a_{condition}'] = (
        (
            quin['FKP14A'].isin(codes) |
            quin['FKL15A'].isin(codes) |
            quin['FKL17A'].isin(codes) |
            quin['fkl24a'].isin(codes)
        ) & 
        ((quin['combined_date'] - quin['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
quin['PSTV18'] = quin['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
quin['ref_year'] = quin['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
quin['mort'] = (quin['PSTV18'] == quin['ref_year']).astype(int)
quin['1ymort'] = (abs(quin['PSTV18'] - quin['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
quin.drop(columns=['ref_year'], inplace=True)


In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
quin = quin.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
quin.to_csv("quin.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
go = quin.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'

}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in go.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_go = go.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_go

In [ ]:
fin_go.to_csv("5th.csv",index=False)

In [ ]:
#enem

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
roku = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 3
valid_pstv01 = roku.loc[roku['n_preg'] == 6, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
roku = roku[roku['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = roku.loc[roku['n_preg'] == 6, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
roku['ref'] = roku['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
roku = roku.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
roku['combined_date'] = pd.to_datetime(roku['combined_date'], errors='coerce')
roku['ref'] = pd.to_datetime(roku['ref'], errors='coerce')

In [ ]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    roku[f'b_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & (roku['combined_date'] <= roku['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    roku[f'b_{condition}'] = (
      (  
        roku['FKP14A'].str.contains(regex, na=False) |
        roku['FKL15A'].str.contains(regex, na=False) |
        roku['FKL17A'].str.contains(regex, na=False) |
        roku['fkl24a'].str.contains(regex, na=False)
      ) & (roku['combined_date'] <= roku['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    roku[f'b_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & 
        ((roku['ref'] - roku['combined_date']).dt.days > 30)
    ).astype(int)

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    roku[f'c_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & 
        ((roku['combined_date'] - roku['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    roku[f'c_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & 
        ((roku['combined_date'] - roku['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    roku[f'a_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & (roku['combined_date'] > roku['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    roku[f'a_{condition}'] = (
      (  
        roku['FKP14A'].str.contains(regex, na=False) |
        roku['FKL15A'].str.contains(regex, na=False) |
        roku['FKL17A'].str.contains(regex, na=False) |
        roku['fkl24a'].str.contains(regex, na=False)
      ) & (roku['combined_date'] > roku['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    roku[f'a_{condition}'] = (
        (
            roku['FKP14A'].isin(codes) |
            roku['FKL15A'].isin(codes) |
            roku['FKL17A'].isin(codes) |
            roku['fkl24a'].isin(codes)
        ) & 
        ((roku['combined_date'] - roku['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
roku = roku.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
roku['PSTV18'] = roku['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
roku['ref_year'] = roku['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
roku['mort'] = (roku['PSTV18'] == roku['ref_year']).astype(int)
roku['1ymort'] = (abs(roku['PSTV18'] - roku['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
roku.drop(columns=['ref_year'], inplace=True)


In [ ]:
roku.to_csv("roku.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
sext = roku.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'

}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in sext.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_sext = sext.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_sext

In [ ]:
fin_sext.to_csv("6th.csv",index=False)

In [ ]:
#pitu

In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
sept = pd.read_csv("pregnancy new.csv")

In [ ]:
# Filter PSTV01 yang memiliki setidaknya satu baris dengan n_preg = 3
valid_pstv01 = sept.loc[sept['n_preg'] == 7, 'PSTV01'].unique()

# Retain only rows with PSTV01 in valid_pstv01
sept = sept[sept['PSTV01'].isin(valid_pstv01)].reset_index(drop=True)

In [ ]:
# Buat kolom 'ref' dengan nilai 'dopt' di mana n_preg == 2 untuk setiap PSTV01
ref_values = sept.loc[sept['n_preg'] == 7, ['PSTV01', 'dopt']].set_index('PSTV01')

# Gabungkan nilai 'ref' ke tabel primi berdasarkan PSTV01
sept['ref'] = sept['PSTV01'].map(ref_values['dopt'])

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKP02', 'FKL02', 'FKL03', 'FKL09', 'FKL16', 'FKL18', 'FKL04', 'fkl24', 'FKP03', 'FKP04', 'FKP15', 'doa', 'fin_a', 'dop', 'fin_p', 'dopt', 'fin_g']

# Hapus kolom yang tidak diperlukan
sept = sept.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Pastikan kolom 'combined_date' dan 'ref' adalah datetime
sept['combined_date'] = pd.to_datetime(sept['combined_date'], errors='coerce')
sept['ref'] = pd.to_datetime(sept['ref'], errors='coerce')

In [ ]:
# Dictionary untuk setiap kondisi diagnosis
conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
    
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    sept[f'b_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & (sept['combined_date'] <= sept['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    sept[f'b_{condition}'] = (
      (  
        sept['FKP14A'].str.contains(regex, na=False) |
        sept['FKL15A'].str.contains(regex, na=False) |
        sept['FKL17A'].str.contains(regex, na=False) |
        sept['fkl24a'].str.contains(regex, na=False)
      ) & (sept['combined_date'] <= sept['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Kehamilan lalu 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    sept[f'b_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & 
        ((sept['ref'] - sept['combined_date']).dt.days > 30)
    ).astype(int)


In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    sept[f'c_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & 
        ((sept['combined_date'] - sept['ref']).dt.days.between(-30, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int) 

In [ ]:
# Kehamilan sekarang 
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    sept[f'c_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & 
        ((sept['combined_date'] - sept['ref']).dt.days.between(-270, 28))  # Rentang 9 bulan (-270 hari) hingga 28 hari
    ).astype(int)

In [ ]:
#all after
# Dictionary untuk setiap kondisi diagnosis
new_conditions = {
    'typhoid': ['A01'],
    'cholera': ['A00'],
    'v_age': ['A08'],
    'b_age': ['A00', 'A02', 'A03', 'A04', 'A05'],
    'p_age': ['A06', 'A07'],
    'tb': ['A15', 'A16', 'A17', 'A18', 'A19'],
    'myco': ['A30', 'A31'],
    'lepto': ['A27'],
    'std': ['A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A63', 'A64'],
    'torch': ['B58', 'B06', 'B25', 'B00', 'A60'],
    'v_skin': ['B01', 'B02', 'B03', 'B04', 'B05', 'B07', 'B08', 'B09'],
    'hepatitis': ['B15', 'B16', 'B17', 'B18', 'B19'],
    'hiv': ['B20', 'B21', 'B22', 'B23', 'B24'],
    'sepsis': ['A40', 'A41'],
    'dm': ['E10', 'E11', 'E12', 'E13', 'E14', 'O24'],
    'abortive': ['O00', 'O01', 'O02', 'O03', 'O04', 'O05', 'O06', 'O07', 'O08'],
    'malnut' : ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46'],
    'nutri' : ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64'],
    'obese' : ['E66'],
    'substance': ['F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19'],
    'schizo': ['F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29'],
    'neurot': ['F40', 'F41', 'F42', 'F43', 'F44', 'F48', 'F45'],
    'infla_cns': ['G00', 'G01', 'G02', 'G03', 'G04', 'G08', 'G05', 'G06', 'G07', 'G09'],
    'neu_deg': ['G10', 'G11', 'G12', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G30', 'G31', 'G32', 'G35', 'G36', 'G37'],
    'headache': ['G43', 'G44'],
    'neuropathy': ['G50', 'G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60', 'G61', 'G62', 'G63', 'G64'],
    'rhd': ['I05', 'I06', 'I07', 'I08', 'I09'],
    'ht': ['I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'O10', 'O13', 'O16'],
    'isch': ['I20', 'I21', 'I22', 'I23', 'I24', 'I25'],
    'phd': ['I26', 'I27', 'I28'],
    'carditis': ['I30', 'I32', 'I33', 'I38', 'I39', 'I40', 'I41'],
    'cmp': ['I42', 'I43'],
    'arrythmia': ['I44', 'I45', 'I47', 'I48', 'I49'],
    'hf': ['I50'],
    'stroke': ['I60', 'I61', 'I62', 'I63', 'I64', 'I69'],
    'artery': ['I70', 'I71', 'I72', 'I73', 'I74', 'I77', 'I78', 'I79'],
    'vein': ['I80', 'I81', 'I82', 'I83', 'I85', 'I86', 'I87', 'I88', 'I89'],
    'urti': ['J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11'],
    'lrti': ['J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22'],
    'chronic_res': ['J35', 'J37', 'J40', 'J41', 'J42', 'J43', 'J44', 'J45'],
    'pul_edema': ['J81'],
    'pleura': ['J90', 'J91', 'J92', 'J93', 'J94'],
    'oral': ['K00', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14'],
    'gastritis': ['K22', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30'],
    'hernia': ['K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46'],
    'intestinal': ['K50', 'K51', 'K52', 'K56', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63'],
    'hemorrh': ['K64'],
    'periton': ['K65'],
    'liver_fail': ['K72'],
    'liver': ['K70', 'K71', 'K73', 'K74', 'K75', 'K76'],
    'gallbladder': ['K80', 'K81', 'K82', 'K83'],
    'pancreas': ['K85', 'K86'],
    'bullous': ['L10', 'L11', 'L12', 'L13', 'L14'],
    'atopic': ['L20'],
    'dermatitis': ['L21', 'L23', 'L25', 'L26', 'L27', 'L28', 'L30'],
    'urticaria': ['L50'],
    'urolith' : ['N20', 'N21', 'N22'],
    'uti' : ['N30', 'N34', 'N39'],
    'endomet' : ['N80'],
    'femgen' : ['N81', 'N82', 'N83', 'N84', 'N85', 'N86', 'N87', 'N88', 'N89', 'N90'],
    'hypomen' : ['N91'],
    'menorrh' : ['N92'], 
    'dysmen' : ['N94'],
    'sga' : ['P05'], 
    'lga' : ['P08'],
     
}

# Iterasi setiap kondisi untuk membuat kolom baru dengan prefix 'b_'
for condition, codes in conditions.items():
    sept[f'a_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & (sept['combined_date'] > sept['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
new_conditions_regex = {
    'arthropathy': r'M(0[0-9]|1[0-9]|2[0-5])',
    'sysconn': r'M3[0-6]',
    'dorsopathy': r'M4[0-9]|M5[0-4]',
    'muscle_dis': r'M6[0-3]',
    'synov_dis': r'M6[5-8]',
    'soft_dis': r'M8[0-9]|M9[0-4]',
    'renal_dis': r'N0[0-9]|N1[0-6]',
    'renal_fail': r'N1[7-9]',
    'breast_dis': r'N6[0-4]',
    'pid': r'N7[0-7]',
    'poison': r'T3[6-9]|T4[0-9]|T50',
    'toxic': r'T5[1-9]|T6[0-5]'
}

for condition, regex in conditions_regex.items():
    sept[f'a_{condition}'] = (
      (  
        sept['FKP14A'].str.contains(regex, na=False) |
        sept['FKL15A'].str.contains(regex, na=False) |
        sept['FKL17A'].str.contains(regex, na=False) |
        sept['fkl24a'].str.contains(regex, na=False)
      ) & (sept['combined_date'] > sept['ref'])  # Syarat tambahan
    ).astype(int)


In [ ]:
# Dictionary baru khusus untuk kondisi tambahan ini
new_conditions = {
    'preecl': ['O11', 'O14'],
    'ecl': ['O15'],
    'earlyhemo': ['O20'],
    'heg': ['O21'],
    'venpreg': ['O22'],
    'utipreg': ['O23'],
    'malpreg': ['O25'],
    'multigest': ['O30'],
    'malpresent': ['O32'],
    'disprop': ['O33'],
    'abnorpelv': ['O34'],
    'fetalprob': ['O35', 'O36'],
    'polyhydra': ['O40'],
    'abnamnio': ['O41'],
    'prom': ['O42'],
    'placental': ['O43'],
    'previa': ['O44'],
    'abrupt': ['O45'],
    'anh': ['O46'],
    'prolong': ['O48'],
    'preterm': ['O60'],
    'fail': ['O61'],
    'abnforce': ['O62'],
    'long': ['O63'],
    'obspelvic': ['O65', 'O66'],
    'malpres': ['O64'],
    'iph': ['O67'],
    'distress': ['O68'],
    'umbilical': ['O69'],
    'laceration': ['O70'],
    'obstrau': ['O71'],
    'pph': ['O72'],
    'retained': ['O73'],
    'normal': ['O80'],
    'instrum': ['O81'],
    'caesar': ['O82'],
    'assisted': ['O83'],
    'multiple': ['O84']
}

# Iterasi untuk kondisi baru
for condition, codes in new_conditions.items():
    sept[f'a_{condition}'] = (
        (
            sept['FKP14A'].isin(codes) |
            sept['FKL15A'].isin(codes) |
            sept['FKL17A'].isin(codes) |
            sept['fkl24a'].isin(codes)
        ) & 
        ((sept['combined_date'] - sept['ref']).dt.days > 28)  # Syarat tambahan
    ).astype(int)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['FKL15A', 'FKL17A', 'fkl24a', 'FKP14A']

# Hapus kolom yang tidak diperlukan
sept = sept.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Ganti NaN di PSTV18 dengan nilai default (misalnya, 0 atau tahun tertentu)
sept['PSTV18'] = sept['PSTV18'].fillna(0).astype(int)

# Ekstrak tahun dari kolom 'ref'
sept['ref_year'] = sept['ref'].dt.year

# Tambahkan kolom kategori ke dalam tabel primi
sept['mort'] = (sept['PSTV18'] == sept['ref_year']).astype(int)
sept['1ymort'] = (abs(sept['PSTV18'] - sept['ref_year']) == 1).astype(int)

# Hapus kolom 'ref_year' setelah selesai digunakan
sept.drop(columns=['ref_year'], inplace=True)


In [ ]:
sept.to_csv("sept.csv",index=False)

In [ ]:
# Daftar kolom yang ingin dihapus
columns_to_drop = ['combined_date', 'ref', 'PSTV18']

# Hapus kolom yang tidak diperlukan
nana = sept.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
aggregation_rules = {
    'subsid': 'max',  # Take the largest value for subsid
    'age': 'min',     # Take the smallest value for age
    'dom': 'min',     # Take the smallest value for dom
    'n_preg' : 'min',
    'outcome': 'max', # Take the smallest value for outcome
    'mort' : 'max',
    '1ymort' : 'max'
}

# Add rules for columns starting with 'b_' or 'a_' to cap their sum at 1
for col in nana.columns:
    if col.startswith('b_') or col.startswith('a_') or col.startswith('c_'):
        aggregation_rules[col] = lambda x: min(x.sum(), 1)

# Perform the groupby and aggregation
fin_nana = nana.groupby('PSTV01').agg(aggregation_rules).reset_index()

In [ ]:
fin_nana

In [ ]:
fin_nana.to_csv("7th.csv",index=False)

In [ ]:
#Join all seven

In [ ]:
one = pd.read_csv("1st_washed.csv")
two = pd.read_csv("2nd.csv")
thr = pd.read_csv("3rd.csv")
fou = pd.read_csv("4th.csv")
fiv = pd.read_csv("5th.csv")
six = pd.read_csv("6th.csv")
sev = pd.read_csv("7th.csv")

In [ ]:
# Daftar tabel
tables = {'one': one, 'two': two, 'thr': thr, 'fou': fou, 'fiv': fiv, 'six': six, 'sev': sev}

# Mengecek jumlah kolom di setiap tabel
columns_count = {name: df.shape[1] for name, df in tables.items()}

# Menampilkan jumlah kolom setiap tabel
for name, count in columns_count.items():
    print(f"Tabel {name} memiliki {count} kolom.")

# Mengecek apakah semua tabel memiliki jumlah kolom yang sama
all_same = len(set(columns_count.values())) == 1
if all_same:
    print("\nSemua tabel memiliki jumlah kolom yang sama.")
else:
    print("\nTabel memiliki jumlah kolom yang berbeda.")


In [ ]:
one['n_preg'] = 1
two['n_preg'] = 2
thr['n_preg'] = 3
fou['n_preg'] = 4
fiv['n_preg'] = 5
six['n_preg'] = 6
sev['n_preg'] = 7

In [ ]:
all = pd.concat([one, two, thr, fou, fiv, six, sev], axis=0, ignore_index=True)

In [ ]:
all.to_csv("final_set.csv",index=False)